In [35]:
from preprocessing.preprocessing import DataPreprocessing
from training.train import DataTraining
from mining.routes import RoutesMining
from hmmlearn import hmm
import pandas as pd
from joblib import dump, load
import numpy as np
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from generate.generate import GenerateData

In [36]:
preprocessing = DataPreprocessing("data/train_trips.csv")
#df = preprocessing.create_formatted()

### TEST FÜR 1 TRIP

In [37]:
trip = df.loc[df['TripID'] == 1]

In [38]:
trip

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,ArrivalHour,TripID,on_train,going_next_station
0,247283,Wedel,89,0,2016-12-10 11:39:06,2016-12-10 11:43:29,WL,S1,2016-12-10,11,1,89,89
1,247283,Rissen,17,2,2016-12-10 11:47:17,2016-12-10 11:47:50,RI,S1,2016-12-10,11,1,15,104
2,247283,Sülldorf,14,8,2016-12-10 11:50:32,2016-12-10 11:52:07,SDF,S1,2016-12-10,11,1,6,110
3,247283,Iserbrook,6,0,2016-12-10 11:53:21,2016-12-10 11:53:57,IS,S1,2016-12-10,11,1,6,116
4,247283,Blankenese,12,7,2016-12-10 11:55:54,2016-12-10 11:57:31,B,S1,2016-12-10,11,1,5,121
5,247283,Hochkamp,5,3,2016-12-10 11:59:45,2016-12-10 12:00:22,HPS,S1,2016-12-10,11,1,2,123
6,247283,Klein Flottbek,3,0,2016-12-10 12:01:49,2016-12-10 12:02:26,FB,S1,2016-12-10,12,1,3,126
7,247283,Othmarschen,46,7,2016-12-10 12:03:58,2016-12-10 12:04:49,OH,S1,2016-12-10,12,1,39,165
8,247283,Bahrenfeld,9,5,2016-12-10 12:06:28,2016-12-10 12:07:10,BAF,S1,2016-12-10,12,1,4,169
9,247283,Altona,136,53,2016-12-10 12:09:59,2016-12-10 12:11:50,AS,S1,2016-12-10,12,1,83,252


In [39]:
trip['percent_leaving'] = (trip['Alightings'] / trip['going_next_station'].shift(1).fillna(0)).fillna(0)

C:\Users\StephanosFidan\AppData\Local\Temp\ipykernel_5452\238336105.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip['percent_leaving'] = (trip['Alightings'] / trip['going_next_station'].shift(1).fillna(0)).fillna(0)


In [40]:
trip['percent_staying'] = None

# Set the first row for the transition from Station 1 to Station 1 to 1
trip.loc[0, 'percent_staying'] = 1

# Calculate the percentage of staying for the rest of the DataFrame
for i in range(1, len(trip)):
    trip.loc[i, 'percent_staying'] = trip.loc[i - 1, 'percent_staying'] * (1 - trip.loc[i - 1, 'percent_leaving'])

# Handle NaN values if necessary (e.g., fill with 0 or drop)
trip.fillna(0, inplace=True)

trip['probability_leaving'] =  trip['percent_staying'].shift(1).fillna(1) - trip['percent_staying']

# View the DataFrame
trip

C:\Users\StephanosFidan\AppData\Local\Temp\ipykernel_5452\2468732525.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip['percent_staying'] = None
C:\Users\StephanosFidan\AppData\Local\Temp\ipykernel_5452\2468732525.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip.fillna(0, inplace=True)
C:\Users\StephanosFidan\AppData\Local\Temp\ipykernel_5452\2468732525.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,ArrivalHour,TripID,on_train,going_next_station,percent_leaving,percent_staying,probability_leaving
0,247283,Wedel,89,0,2016-12-10 11:39:06,2016-12-10 11:43:29,WL,S1,2016-12-10,11,1,89,89,0.000000,1,0.0
1,247283,Rissen,17,2,2016-12-10 11:47:17,2016-12-10 11:47:50,RI,S1,2016-12-10,11,1,15,104,0.022472,1.0,0.0
2,247283,Sülldorf,14,8,2016-12-10 11:50:32,2016-12-10 11:52:07,SDF,S1,2016-12-10,11,1,6,110,0.076923,0.977528,0.022472
3,247283,Iserbrook,6,0,2016-12-10 11:53:21,2016-12-10 11:53:57,IS,S1,2016-12-10,11,1,6,116,0.000000,0.902334,0.075194
4,247283,Blankenese,12,7,2016-12-10 11:55:54,2016-12-10 11:57:31,B,S1,2016-12-10,11,1,5,121,0.060345,0.902334,0.0
5,247283,Hochkamp,5,3,2016-12-10 11:59:45,2016-12-10 12:00:22,HPS,S1,2016-12-10,11,1,2,123,0.024793,0.847882,0.054451
6,247283,Klein Flottbek,3,0,2016-12-10 12:01:49,2016-12-10 12:02:26,FB,S1,2016-12-10,12,1,3,126,0.000000,0.826861,0.021022
7,247283,Othmarschen,46,7,2016-12-10 12:03:58,2016-12-10 12:04:49,OH,S1,2016-12-10,12,1,39,165,0.055556,0.826861,0.0
8,247283,Bahrenfeld,9,5,2016-12-10 12:06:28,2016-12-10 12:07:10,BAF,S1,2016-12-10,12,1,4,169,0.030303,0.780924,0.045937
9,247283,Altona,136,53,2016-12-10 12:09:59,2016-12-10 12:11:50,AS,S1,2016-12-10,12,1,83,252,0.313609,0.75726,0.023664


### NUMERATE STATIONS WITHIN A TRIP

In [41]:
def numerate_stations(group):
    i = 0
    for idx in range(0, len(group)-1):
        i = i + 1
        group.iloc[idx, group.columns.get_loc('station_id')] = i 

    return group

In [42]:
#df = df.groupby(['TripID']).apply(numerate_stations)

### Aggregate DATA - Average Boardings, Alightings, Next Station

In [43]:
#aggregated_data = df.groupby('Station',  sort=False).mean(['Boardings', 'Alightings','going_next_station']).reset_index()

In [44]:
#aggregated_data.iloc[3:len(aggregated_data)]

### Try to iterate over dataset - WIP ...

In [94]:
n_stations = len(aggregated_data)
transition_matrix = np.zeros((n_stations, n_stations))

In [177]:
def calculate_probabilities(fahrt):
    #id = fahrt['TripID'].unique()
    #id = id[0]

     
    for idx in range(0, len(fahrt)-1):  # 1|len(fahrt)-1

        st = str(idx + 1)

        #ps = percentage staying
        fahrt.loc[0:idx, 'ps_' + st] = 1 #set preceding stations to 100%
        
        j = idx + 1
        for j in range(j, len(fahrt)):
            fahrt.loc[j, 'ps_' + st] = fahrt.loc[j - 1, 'ps_' + st] * (1 - fahrt.loc[j, 'percent_leaving']) # ps = ps(1) * ( 1 - percentatge leaving )

        # # Handle NaN values if necessary (e.g., fill with 0 or drop)
        fahrt.fillna(0)
        
        #pl = propability leaving
        fahrt['pl_' + st ] =  (fahrt['ps_' + st].shift(1) - fahrt['ps_' + st]).fillna(0)
                
        # fahrt['pl_' + str(idx+1)] =  fahrt['ps_' + str(idx+1)].shift(idx + 1).fillna(idx + 1) - fahrt['ps_' + str(idx + 1)]
    return fahrt



##### gibt es irgendein Fehler...das problem mit der zweiten Runde..ich erstelle dynamisch Spalte für Wahrescheinlichkeiten für jeden Trip. Allerdings muss man auch die Wahrscheinlichkeiten für den Trip zurück betrachten. Von 2 nach 1 und nach 0...

In [ ]:
#test = trip.groupby(['TripID']).apply(calculate_probabilities)
#pl_cols = test.iloc[:, :4]  # Select all rows and first two columns
# last_five_columns = test.iloc[:, -5:]  # Select all rows and last five columns

# # Concatenate horizontally
# result = pd.concat([first_two_columns, last_five_columns], axis=1)

# # Print the result
# print(result)

In [182]:
def get_pl_columns(trip):
    dfpl = pd.DataFrame()
    for x in range(1, len(trip.columns) + 1):  # Assuming you want to iterate based on the number of columns
        column_name = 'pl_' + str(x)
        if column_name in trip.columns:  # Check if column exists
            dfpl[column_name] = trip[column_name]  # Add column to dfpl
    return dfpl

In [184]:
df_result = get_pl_columns(test) 

In [185]:
df_result

pl_1      pl_2      pl_3      pl_4      pl_5      pl_6  \
TripID                                                                  
1      0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
       1   0.022472  0.000000  0.000000  0.000000  0.000000  0.000000   
       2   0.075194  0.076923  0.000000  0.000000  0.000000  0.000000   
       3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
       4   0.054451  0.055703  0.060345  0.060345  0.000000  0.000000   
       5   0.021022  0.021505  0.023297  0.023297  0.024793  0.000000   
       6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
       7   0.045937  0.046993  0.050909  0.050909  0.054178  0.055556   
       8   0.023664  0.024208  0.026226  0.026226  0.027910  0.028620   
       9   0.237484  0.242943  0.263188  0.263188  0.280090  0.287211   
       10  0.002063  0.002110  0.002286  0.002286  0.002433  0.002494   
       11  0.040926  0.041867  0.045356  0.045356  0.048268  0.049496   
       12  0.078551  0.080357  0.087053  0.087053  0.092644  0.094999   
       13  0.017246  0.017643  0.019113  0.019113  0.020341  0.020858   
       14  0.200440  0.205048  0.222135  0.222135  0.236401  0.242411   
       15  0.123607  0.126448  0.136986  0.136986  0.145783  0.149489   
       16  0.002997  0.003066  0.003321  0.003321  0.003535  0.003625   
       17  0.004231  0.004328  0.004689  0.004689  0.004990  0.005117   
       18  0.006960  0.007120  0.007713  0.007713  0.008209  0.008417   
       19  0.008717  0.008917  0.009660  0.009660  0.010281  0.010542   
       20  0.003094  0.003165  0.003429  0.003429  0.003649  0.003742   
       21  0.005596  0.005725  0.006202  0.006202  0.006600  0.006768   
       22  0.001536  0.001572  0.001702  0.001702  0.001812  0.001858   
       23  0.004465  0.004567  0.004948  0.004948  0.005266  0.005399   
       24  0.009117  0.009327  0.010104  0.010104  0.010753  0.011026   
       25  0.010229  0.010464  0.011336  0.011336  0.012064  0.012371   

               pl_7      pl_8      pl_9     pl_10  ...     pl_16     pl_17  \
TripID                                             ...                       
1      0   0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
       1   0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
       2   0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
       3   0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
       4   0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
       5   0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
       6   0.000000  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
       7   0.055556  0.000000  0.000000  0.000000  ...  0.000000  0.000000   
       8   0.028620  0.030303  0.000000  0.000000  ...  0.000000  0.000000   
       9   0.287211  0.304106  0.313609  0.000000  ...  0.000000  0.000000   
       10  0.002494  0.002641  0.002724  0.003968  ...  0.000000  0.000000   
       11  0.049496  0.052407  0.054045  0.078738  ...  0.000000  0.000000   
       12  0.094999  0.100587  0.103731  0.151125  ...  0.000000  0.000000   
       13  0.020858  0.022085  0.022775  0.033181  ...  0.000000  0.000000   
       14  0.242411  0.256671  0.264692  0.385628  ...  0.000000  0.000000   
       15  0.149489  0.158283  0.163229  0.237808  ...  0.000000  0.000000   
       16  0.003625  0.003838  0.003958  0.005766  ...  0.052632  0.000000   
       17  0.005117  0.005418  0.005587  0.008140  ...  0.074303  0.078431   
       18  0.008417  0.008913  0.009191  0.013390  ...  0.122229  0.129020   
       19  0.010542  0.011162  0.011511  0.016771  ...  0.153083  0.161588   
       20  0.003742  0.003962  0.004086  0.005953  ...  0.054341  0.057360   
       21  0.006768  0.007166  0.007390  0.010766  ...  0.098277  0.103736   
       22  0.001858  0.001967  0.002029  0.002955  ...  0.026978  0.028477   
       23  0.005399 

Ab hier Stephanos

'2'

In [82]:
test

TrainID              Station  Boardings  Alightings  \
TripID                                                          
1      0   247283                Wedel         89           0   
       1   247283               Rissen         17           2   
       2   247283             Sülldorf         14           8   
       3   247283            Iserbrook          6           0   
       4   247283           Blankenese         12           7   
       5   247283             Hochkamp          5           3   
       6   247283       Klein Flottbek          3           0   
       7   247283          Othmarschen         46           7   
       8   247283           Bahrenfeld          9           5   
       9   247283               Altona        136          53   
       10  247283          Königstraße          2           1   
       11  247283           Reeperbahn         28          20   
       12  247283      Landungsbrücken         36          43   
       13  247283      Stadthausbrücke          6          11   
       14  247283        Jungfernstieg         12         131   
       15  247283         Hauptbahnhof         54          89   
       16  247283       Berliner Tor 1         12           5   
       17  247283             Landwehr          6           8   
       18  247283          Hasselbrook         17          14   
       19  247283  Wandsbeker Chaussee         17          21   
       20  247283       Friedrichsberg          4           9   
       21  247283              Barmbek         22          17   
       22  247283            Alte Wöhr          3           6   
       23  247283            Rübenkamp          9          18   
       24  247283             Ohlsdorf         36          41   
       25  247283              Airport          0          82   

                       Arrival            Departure StationNameShort sBahnID  \
TripID                                                                         
1      0   2016-12-10 11:39:06  2016-12-10 11:43:29               WL      S1   
       1   2016-12-10 11:47:17  2016-12-10 11:47:50               RI      S1   
       2   2016-12-10 11:50:32  2016-12-10 11:52:07              SDF      S1   
       3   2016-12-10 11:53:21  2016-12-10 11:53:57               IS      S1   
       4   2016-12-10 11:55:54  2016-12-10 11:57:31                B      S1   
       5   2016-12-10 11:59:45  2016-12-10 12:00:22              HPS      S1   
       6   2016-12-10 12:01:49  2016-12-10 12:02:26               FB      S1   
       7   2016-12-10 12:03:58  2016-12-10 12:04:49               OH      S1   
       8   2016-12-10 12:06:28  2016-12-10 12:07:10              BAF      S1   
       9   2016-12-10 12:09:59  2016-12-10 12:11:50               AS      S1   
       10  2016-12-10 12:13:20  2016-12-10 12:13:58               KS      S1   
       11  2016-12-10 12:15:06  2016-12-10 12:16:09              RES      S1   
       12  2016-12-10 12:17:21  2016-12-10 12:18:07              LAS      S1   
       13  2016-12-10 12:19:29  2016-12-10 12:20:09              SHS      S1   
       14  2016-12-10 12:21:02  2016-12-10 12:21:54              JUS      S1   
       15  2016-12-10 12:23:47  2016-12-10 12:24:51               HS      S1   
       16  2016-12-10 12:26:25  2016-12-10 12:27:21             BTSL      S1   
       17  2016-12-10 12:28:49  2016-12-10 12:29:28              LAN      S1   
       18  2016-12-10 12:30:47  2016-12-10 12:31:23              HSB      S1   
       19  2016-12-10 12:32:13  2016-12-10 12:33:19              WCH      S1   
       20  2016-12-10 12:34:22  2016-12-10 12:35:03              FBG      S1   
       21  2016-12-10 12:36:40  2016-12-10 12:37:39              BAP      S1   
       22  2016-12-10 12:39:04  2016-12-10 12:39:41               AW      S1   
       23  2016-12-10 12:40:38  2016-12-10 12:41:21               RP      S1   
       24  2016-12-10 12:43:12  2016-12-10 12:45:13              OPS      S1   
       25  2016-12-10 12:48:1

In [25]:
print(df['TripID'].unique())

[    1     2     3 ... 10376 10377 10378]


In [52]:
trip.head(50)

,TrainID,Station,Boardings,Alightings,Arrival,Departure,StationNameShort,sBahnID,ArrivalDate,ArrivalHour,TripID,on_train,going_next_station,percent_leaving,percent_staying,probability_leaving
0,247283,Wedel,89,0,2016-12-10 11:39:06,2016-12-10 11:43:29,WL,S1,2016-12-10,11,1,89,89,0.000000,1,0.0
1,247283,Rissen,17,2,2016-12-10 11:47:17,2016-12-10 11:47:50,RI,S1,2016-12-10,11,1,15,104,0.022472,1.0,0.0
2,247283,Sülldorf,14,8,2016-12-10 11:50:32,2016-12-10 11:52:07,SDF,S1,2016-12-10,11,1,6,110,0.076923,0.977528,0.022472
3,247283,Iserbrook,6,0,2016-12-10 11:53:21,2016-12-10 11:53:57,IS,S1,2016-12-10,11,1,6,116,0.000000,0.902334,0.075194
4,247283,Blankenese,12,7,2016-12-10 11:55:54,2016-12-10 11:57:31,B,S1,2016-12-10,11,1,5,121,0.060345,0.902334,0.0
5,247283,Hochkamp,5,3,2016-12-10 11:59:45,2016-12-10 12:00:22,HPS,S1,2016-12-10,11,1,2,123,0.024793,0.847882,0.054451
6,247283,Klein Flottbek,3,0,2016-12-10 12:01:49,2016-12-10 12:02:26,FB,S1,2016-12-10,12,1,3,126,0.000000,0.826861,0.021022
7,247283,Othmarschen,46,7,2016-12-10 12:03:58,2016-12-10 12:04:49,OH,S1,2016-12-10,12,1,39,165,0.055556,0.826861,0.0
8,247283,Bahrenfeld,9,5,2016-12-10 12:06:28,2016-12-10 12:07:10,BAF,S1,2016-12-10,12,1,4,169,0.030303,0.780924,0.045937
9,247283,Altona,136,53,2016-12-10 12:09:59,2016-12-10 12:11:50,AS,S1,2016-12-10,12,1,83,252,0.313609,0.75726,0.023664


In [ ]:
trip